In [1]:
import pandas as pd
import numpy as np
import re 

In [2]:
# Path to data
file_path = 'https://s3-us-west-2.amazonaws.com/com.guild.us-west-2.public-data/project-data/movies_metadata_sample.csv.gz'
# Loading data into a DataFrame 
df = pd.read_csv(file_path, compression='gzip')

In [3]:
# Checking out first few rows of the DataFrame
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [4]:
# Removing unnecessary columns
new_df = df.filter(['genres','release_date'], axis=1)

In [5]:
# Formatting release_date year for easier grouping
new_df['year'] = new_df['release_date'].astype(str).str.extract('(\d{4})')
new_df.drop('release_date', axis=1, inplace=True)
new_df.head()

,genres,year
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995
4,"[{'id': 35, 'name': 'Comedy'}]",1995


In [6]:
# Cleaning genres column
def clean_genres(genres_col):
    return re.findall(r'\b[A-Z]\w+', genres_col)
    
# Apply function to entire column
new_df['genres'] = new_df['genres'].apply(lambda x: clean_genres(x))
new_df.head()

,genres,year
0,"[Animation, Comedy, Family]",1995
1,"[Adventure, Fantasy, Family]",1995
2,"[Romance, Comedy]",1995
3,"[Comedy, Drama, Romance]",1995
4,[Comedy],1995


In [7]:
# Making genres column easier to group
# Separate genre array elements while keeping all other row values 
new_df = new_df.explode('genres')
new_df.head()

,genres,year
0,Animation,1995
0,Comedy,1995
0,Family,1995
1,Adventure,1995
1,Fantasy,1995


In [8]:
# Create column for count of each genre per year
final_df = new_df.groupby(['genres', 'year']).size().reset_index()
final_df = final_df.rename(columns = {'genres': 'genre', 0: 'count'})
final_df

,genre,year,count
0,Action,1926,2
1,Action,1927,1
2,Action,1931,1
3,Action,1933,2
4,Action,1935,1
...,...,...,...
1238,Western,1997,1
1239,Western,1998,2
1240,Western,1999,4
1241,Western,2000,4


In [9]:
# Write data to new csv file
final_df.to_csv('clean_movies.csv', index=False)